In [1]:
with open('imports.py', 'r') as file:
    imports_code = file.read()

with open('utils.py', 'r') as file:
    utils_code = file.read()

with open('funcoes.py', 'r') as file:
    funcoes_code = file.read()

exec(imports_code)
exec(utils_code)
exec(funcoes_code)

In [2]:
# Criando uma sessao
session = requests.Session()

# Fazendo várias requisições com a mesma sessão
dados_corredor = get_dados(session, base_url, categoria="Corredor", token=token)
dados_empresa = get_dados(session, base_url, categoria="Empresa", token=token)
dados_posicao = get_dados(session, base_url, categoria="Posicao", token=token)

# Finalizando a sessão quando não for mais necessária
#session.close()

# Gerar variável para salvar no nome do arquivo
agora = datetime.now().strftime("%Y-%m-%d %H%M%S")

# Salvando dados
path_posicao = f"raw\posicao\{agora}.json"
path_corredor = f"raw\corredor\{agora}.json"
path_empresa = f"raw\empresa\{agora}.json"

salvar_json(dados_corredor, path_corredor, 'utf-8')
salvar_json(dados_empresa, path_empresa, 'utf-8')
salvar_json(dados_posicao, path_posicao, 'utf-8')


Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!


'Arquivo salvo com sucesso!'

# Extrair dados de previsao de cada linha

In [9]:
# Extrair a lista de todas as linhas
lista_linhas = list(set([i['cl'] for i in dados_posicao['l']]))

# Lista que vai receber as requisições de previsao de cada linha retornada no metódo de posição 
dados_previsao = []

for i in lista_linhas[:]:
    previsao_linha = get_dados(session, base_url, categoria="Previsao", metodo="Linha", sufix="codigoLinha", parametro=str(i), token=token)
    dados_previsao.append(previsao_linha)

# Salvando dados de previsao de chegada por linha
agora = datetime.now().strftime("%Y-%m-%d %H%M%S")
path_previsao = f"raw\previsao\{agora}.json"
salvar_json(dados_previsao, path_previsao, 'utf-8')


# Finalizando a sessão quando não for mais necessária
session.close()
    

Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com sucesso!
Autenticacao realizada com s

# Transformar chaves em colunas, renomear e salvar em csv

In [3]:
# Trazer nome dos campos com base no catálogo da api
df = pd.read_excel("catálogo api.xlsx", sheet_name="Objetos de retorno")

# Filtrar registros onde Categoria e Método = Posicao
df_filtrado = df[(df['Categoria'] == 'Posicao') & (df['Métodos'] == 'Posicao')]

# Criar o dicionário usando as colunas "Chave" e "Nome Campo"
mapa_chaves = dict(zip(df_filtrado['Chave'], df_filtrado['Nome Campo']))

# Aplicar função para ajustar nome dos dados
dados_renomeados = renomear_chaves(dados_posicao, mapa_chaves)

In [8]:
# Transformar o dicionário renomeado em um DataFrame
linhas_localizadas = dados_renomeados['linhas_localizadas']

# Extraindo informações dos veículos dentro de cada linha localizada
veiculos_lista = []
for linha in linhas_localizadas:
    for veiculo in linha['veiculos_localizados']:
        # Adicionando detalhes da linha junto com os veículos
        veiculo_completo = veiculo.copy()  # Copia as informações do veículo
        veiculo_completo.update({
            'hora_referencia_infos': dados_renomeados['hora_referencia_infos'],
            'letreiro_completo': linha['letreiro_completo'],
            'codigo_linha': linha['codigo_linha'],
            'sentido_linha': linha['sentido_linha'],
            'letreiro_destino_linha': linha['letreiro_destino_linha'],
            'letreiro_origem_linha': linha['letreiro_origem_linha'],
            'quantidade_veiculos_localizados': linha['quantidade_veiculos_localizados']
        })
        veiculos_lista.append(veiculo_completo)

# Criar o DataFrame com todas as informações dos veículos e suas respectivas linhas
df_completo = pd.DataFrame(veiculos_lista)


In [16]:
# Transformar em string e salvar o dataframe
df_completo = df_completo.astype(str)
df_completo.to_csv('cleaned\posicao.csv', index=False, encoding='utf-8')
df_completo.to_excel('cleaned\posicao.xlsx', index=False)